In [ ]:
# Importing libraries for file and image manipulation
from pathlib import Path  # Path manipulation for file handling
from skimage.io import imread  # Image reading functionality
import numpy as np  # Numerical operations
import cv2  # Computer vision library
import matplotlib.pyplot as plt  # Plotting library
import os  # Operating system interaction
import pandas as pd  # Data manipulation and analysis
import time  # Time-related functionality
import gc  # Garbage collection for memory management


In [ ]:
# Define the directory path
directory = r"D:\Lorenzo\ca2\250118"
directory_path = Path(directory)

# Specify the field_of_view  
field_of_view = '2727_3'
filename = field_of_view +'.tif'

# Create the full path for the image
directory_filename = os.path.join(directory_path, filename)

# Read the image using skimage.io.imread
images = imread(directory_filename)

# Extract mKate and gcamp channels from the image
images_mKate = images[:, 1, :, :]
images_gcamp = images[:, 0, :, :]

print(f'image shape: {images.shape}')
print(f'channel shape: {images_gcamp.shape}')

# Free up memory by deleting the original images array
del images

# Perform garbage collection to release any unused memory
gc.collect();


In [ ]:
# Import functions from the custom module binding_box_finder
from binding_box_finder_v17 import blob_maker, binding_box_find_and_display

# Apply blob_maker function to create a binary image with blobs

bit8 = blob_maker(images_gcamp, images_mKate, 30) # The third parameter change the dimention of the blob 

# Use binding_box_find_and_display to find bounding boxes and display them on the original image
# This function returns coordinates and an image with bounding boxes
coord_list, image_boxes = binding_box_find_and_display(bit8, images_gcamp, 50)

In [ ]:
# List of indices to be removed from coord_list
indices_to_remove = [0]

# Use list comprehension to create a new coord_list excluding elements with specified indices
# Only include items in the new list if their index is not in indices_to_remove
coord_list = [item for index, item in enumerate(coord_list) if index not in indices_to_remove]

# Display the updated coord_list
coord_list


In [ ]:
# List of indices to keep in coord_list
indices_to_keep = [3,5]

# Use list comprehension to create a new coord_list with elements from specified indices
# Only include items in the new list if their index is in indices_to_keep
coord_list = [item for index, item in enumerate(coord_list) if index in indices_to_keep]

# Display the updated coord_list
coord_list


In [ ]:
#append a coordinate
coord_to_append = [[30, 560, 200, 200], [560, 560, 200, 200]] #x, y, w, h top-left corner
for c in coord_to_append:
    coord_list.append(c)
coord_list

In [ ]:
# Import the draw_boxes_from_coordinates function from the custom module binding_box_finder
from binding_box_finder_v17 import draw_boxes_from_coordinates as dbfc

# Specify the filename for the image with drawn bounding boxes
filename_boxes = os.path.join(directory_path, 'boxes_' + filename)


# Use the draw_boxes_from_coordinates function to draw bounding boxes on the first image in images_gcamp
# The function takes coord_list, the image, and the filename as parameters
starting_n_worm = int(input('What is the starting number?')) #Starting number: number to start numbering the worms
_ = dbfc(coord_list, images_gcamp[0], filename_boxes, starting_n_worm)


In [ ]:
# Import functions from the custom module get_intensity_library_opt_v17
from get_RIS_intensity_library_v26 import get_single_worm, get_thresholded_image_yan, get_RIS_intensities, single_frame_analysis, over_time_analysis, multiple_worm_analysis

# Import necessary libraries
import time as t  # Import time library and alias it as 't'
from IPython.display import display, clear_output  # Import display and clear_output functions from IPython

# Record the start time for performance measurement
start_time = t.time()

# List to store dictionaries containing worm analysis results
worm_dict_list = []

# Parameters for analysis
k_blur = 1.5 # Blur parameter
k_blur_analysis = 1.5

replicate = input('Which replicate is this?')

timerange = range(0, len(images_gcamp))  # Time range for analysis: change if you want to analyse a different time window of the movie

# Loop through each coordinate in coord_lst and perform multiple worm analysis
for n_worm, coord in enumerate(coord_list, start=starting_n_worm):
    # Display the updated content
    display(n_worm)
    worm_dict_list.extend(multiple_worm_analysis(coord, n_worm, timerange, images_gcamp, images_mKate, k_blur, k_blur_analysis, replicate))
    
    # Clear the output of the current cell
    clear_output(wait=True)


# Record the end time for performance measurement
end_time = t.time()

# Calculate the elapsed time
ex_time = (end_time - start_time)/60

# Display the elapsed time
print(f'{round(ex_time, 1)} minutes. Last worm: {n_worm}') 


In [ ]:
# Create a DataFrame from the list of dictionaries containing worm analysis results
data_df = pd.DataFrame(worm_dict_list)

data_df['centroid_x'] = pd.to_numeric(data_df['centroid_x'], errors='coerce')
data_df['centroid_y'] = pd.to_numeric(data_df['centroid_y'], errors='coerce')

# Drop rows with NaN values in 'centroid_x' or 'centroid_y'
data_df = data_df.dropna(subset=['centroid_x', 'centroid_y'])

# Group by 'n_worm' and calculate distances within each group
def calculate_distance(group):
    x_squared_diff = (group['centroid_x'] - group['centroid_x'].shift()) ** 2
    y_squared_diff = (group['centroid_y'] - group['centroid_y'].shift()) ** 2
    group['distance'] = (x_squared_diff + y_squared_diff) ** 0.5
    return group

# Apply the function to each group
data_df = data_df.groupby('n_worm', group_keys=False).apply(calculate_distance)

# Define a speed coefficient µm/px * FPS
speed_coefficient = 0.73 * 0.17

# Calculate the speed by multiplying the distance by the speed coefficient
data_df['speed'] = data_df['distance'] * speed_coefficient

# Calculate the normalized intensity by dividing intensity_gcamp by intensity_mKate
data_df['normalised_intensity'] = data_df['intensity_gcamp'] / data_df['intensity_mKate']

data_df['strain']= input('what is the strain name?')

# Display the DataFrame with the added columns
data_df

In [ ]:
from datetime import datetime

# Define info
info = f"The analysis used the following parameter \
\n k_blur={k_blur} \
\n k_blur_analysis={k_blur_analysis}\
\n number of worm= {n_worm}" 

# Generate the current date and time as a string
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")

# Specify the filename for saving the DataFrame to a CSV file
file_to_save = 'intensity_and_speed_df_' + field_of_view + '.csv'
file_to_save_info = 'info_' + field_of_view + '_' + current_time + '.txt'


# Create the full path for saving the CSV file
saving_path = os.path.join(directory_path, file_to_save)


# Save the DataFrame to a CSV file at the specified path
data_df.to_csv(saving_path, index=False)

# Specify the path for the new folder
folder_path = os.path.join(directory_path, 'log_file_folder')
saving_info_path = os.path.join(folder_path, file_to_save_info)
# Create the folder
try:
    os.mkdir(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{folder_path}' already exists.")

# Write to a file using Path
Path(saving_info_path).write_text(info)
